# Design Pattern 1: Hashed Feature
> O padrão aborda três problemas possíveis associados a características categóricas: vocabulário incompleto, tamanho do modelo devido à cardinalidade e início a frio. Ele faz isso agrupando as características categóricas e aceitando o compromisso de colisões na representação dos dados.

### Bibliotecas

In [1]:
import hashlib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### Dados de teste

In [2]:
df = pd.read_csv('data/BrFlights2.csv', encoding='latin1')
print(f'Linhas: {df.shape[0]} | Colunas: {df.shape[1]}')
df.head()

Linhas: 2542519 | Colunas: 21


,Voos,Companhia.Aerea,Codigo.Tipo.Linha,Partida.Prevista,Partida.Real,Chegada.Prevista,Chegada.Real,Situacao.Voo,Codigo.Justificativa,Aeroporto.Origem,...,Estado.Origem,Pais.Origem,Aeroporto.Destino,Cidade.Destino,Estado.Destino,Pais.Destino,LongDest,LatDest,LongOrig,LatOrig
0,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-30T08:58:00Z,2016-01-30T08:58:00Z,2016-01-30T10:35:00Z,2016-01-30T10:35:00Z,Realizado,NaN,Afonso Pena,...,PR,Brasil,Salgado Filho,Porto Alegre,RS,Brasil,-51.175381,-29.993473,-49.172481,-25.532713
1,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-13T12:13:00Z,2016-01-13T12:13:00Z,2016-01-13T21:30:00Z,2016-01-13T21:30:00Z,Realizado,NaN,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473
2,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-29T12:13:00Z,2016-01-29T12:13:00Z,2016-01-29T21:30:00Z,2016-01-29T21:30:00Z,Realizado,NaN,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473
3,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-19T12:13:00Z,2016-01-18T12:03:00Z,2016-01-19T21:30:00Z,2016-01-18T20:41:00Z,Realizado,LIBERACAO SERV. TRAFEGO AEREO/ANTECIPACAO,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473
4,AAL - 203,AMERICAN AIRLINES INC,Internacional,2016-01-30T12:13:00Z,2016-01-30T12:13:00Z,2016-01-30T21:30:00Z,2016-01-30T21:30:00Z,Realizado,NaN,Salgado Filho,...,RS,Brasil,Miami,Miami,N/I,Estados Unidos,-80.287046,25.795865,-51.175381,-29.993473


### Aplicação do padrão

- Passo 1: converte a entrada categórica em uma string;
- Passo 2: aplica um algoritmo de hash;
- Passo 3: calcula o resto ao dividir o resultado do hash pelo número desejado de buckets e garante o valor positivo com o abs.

In [3]:
def hashedFeature(category: str, numBuckets: int) -> int:
    hashResult = int(hashlib.sha256(str(category).encode()).hexdigest(), 16)
    bucketIndex = abs(hashResult) % numBuckets
    return bucketIndex

In [5]:
dados = df[['Voos', 'Companhia.Aerea', 'Codigo.Justificativa']].dropna()
buckets = 10

for column in dados.columns:
    dados[column] = dados[column].apply(lambda x: hashedFeature(x, buckets))
dados.head()

,Voos,Companhia.Aerea,Codigo.Justificativa
3,3,7,3
6,3,7,3
7,3,7,1
9,3,7,0
10,3,7,1


In [6]:
dados['Voos'].value_counts()

5    111336
3    108476
7    108049
2    104246
6    104164
4    102278
9    101585
0     98993
1     98778
8     94402
Name: Voos, dtype: int64